# RF Network - One Port

This is an example of a simulation of a Radio Frequency (RF) network with PathSim.

You can also find a similar example in the [GitHub repository](https://github.com/milanofthe/pathsim/blob/master/examples/example_spectrum_rf_oneport.py).

Let's first make all the necessary import

In [5]:
import matplotlib.pyplot as plt

# Apply PathSim docs matplotlib style for consistent, theme-friendly figures
plt.style.use('../pathsim_docs.mplstyle')

from pathsim import Simulation, Connection
from pathsim.blocks import Spectrum, GaussianPulseSource
from pathsim.solvers import RKBS32

from pathsim.blocks.rf import RFNetwork  # requires the scikit-rf package to be installed

ModuleNotFoundError: No module named 'pathsim.blocks.rf'

In [ ]:
# The RF-Network block is created from a scikit-rf Network object example.
# Here we use a frequency measurement example of a 1-port RF network (included in scikit-rf)
import skrf as rf  # for the example
rfntwk = RFNetwork(rf.data.ring_slot_meas)

Under the hood, scikit-rf performs a Vector Fitting of the frequency data and creates a PathSim State-Space model.

In [ ]:
# Gaussian pulse simulating an impulse response
# Note that the scikit-rf Network object is passed as the 'network' parameter of the block,
# which is convenient to access its frequency data.
src = GaussianPulseSource(f_max=rfntwk.network.frequency.stop)

# Spectrum analyser setup with the start and stop frequencies of the RF network
spc = Spectrum(
    freq=rfntwk.network.f,
    labels=["pulse", "response"]
)

In [ ]:
# create the system connections and simulation setup
sim = Simulation(
    blocks=[src, rfntwk, spc],
    connections=[
        Connection(src, rfntwk, spc[0]),
        Connection(rfntwk, spc[1])
    ],
    tolerance_lte_abs=1e-16, # this is due to the super tiny states
    tolerance_lte_rel=1e-5,  # so error control is dominated by the relative truncation error
    Solver=RKBS32,
)

sim.run(1e-9)

Below, we compare the PathSim's frequency response to the original measurement data and to their Vector Fitted model calculated with scikit-rf. The Vector Fitted model and the PathSim's frequency response are in perfect agreement:

In [ ]:
# model frequency response H(f) recovered from the spectrum block
freq, (G_pulse, G_filt) = spc.read()
H_filt_sim = G_filt / G_pulse

# plot the original S11 data, the vector-fitted model and the recovered frequency response
fig, ax = plt.subplots()
ax.plot(rfntwk.network.f/1e9, abs(rfntwk.network.s[:, 0, 0]), '.', label="S11 measurements", alpha=0.5)
ax.plot(freq/1e9, abs(rfntwk.vf.get_model_response(0, 0, freqs=freq)), lw=2, label="scikit-rf vector-fitting model")
ax.plot(freq/1e9, abs(H_filt_sim), '--', lw=2, label="pathsim impulse response")
ax.set_xlabel("Frequency [GHz]")
ax.set_ylabel("S11 magnitude")
ax.legend()
plt.show()